In [1]:
from facenet_pytorch import MTCNN
import torch
import numpy as np
import cv2
from PIL import Image, ImageDraw
from IPython import display
from tqdm.notebook import tqdm

In [2]:
mtcnn = MTCNN(keep_all=True, device='cuda:0', thresholds=[0.75, 0.85, 0.85])

In [3]:
vid = cv2.VideoCapture('./walking_720.mp4')
vid_len = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
frames = []
for _ in tqdm(range(vid_len)):
    success, frame = vid.read()
    if not success: continue
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(Image.fromarray(frame))


  0%|          | 0/1228 [00:00<?, ?it/s]

In [4]:
display.Video('./walking_720.mp4')

In [5]:
frames_tracked = []
for i, frame in enumerate(frames):
    print("\rTracking frame: {}".format(i+1), end='')
    boxes,_ = mtcnn.detect(frame)
    frame_draw = frame.copy()
    draw = ImageDraw.Draw(frame_draw)
    if boxes is not None:
        for box in boxes:
            draw.rectangle(box.tolist(), outline=(255,0,0), width=6)
    frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))

Tracking frame: 1228

In [6]:
dim = frames_tracked[0].size
fourcc = cv2.VideoWriter_fourcc(*'FMP4')    
video_tracked = cv2.VideoWriter('video_tracked.mp4', fourcc, 25.0, dim)
for frame in frames_tracked:
    video_tracked.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))
video_tracked.release()

OpenCV: FFMPEG: tag 0x34504d46/'FMP4' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
